# Tutorial

In [ ]:
from collections import namedtuple
import matplotlib.pyplot as plt
import numpy as np

from atuproot.AtUproot import AtUproot

In [ ]:
process = AtUproot("output",
                  quiet = False,
                  parallel_mode = 'multiprocessing',
                  process = 0,
                  max_blocks_per_dataset = -1,
                  max_blocks_per_process = -1,
                  nevents_per_block = 1000000,
                  profile = False,
                  profile_out_path = 'profile.txt')

In [ ]:
class TestScribbler(object):
    def __init__(self):
        self.bins = np.linspace(0., 40., 11)

    def begin(self, event):
        self.hist = None
    
    def event(self, event):
        hist, _ = np.histogram(event.pt1, self.bins)
        if self.hist is None:
            self.hist = hist
        else:
            self.hist += hist
            
    def merge(self, other):
        if self.hist is None:
            self = other
            return
        if other.hist is None:
            return
        
        self.hist += other.hist
        
class TestCollector(object):
    def collect(self, dataset_readers_list):
        reader = dataset_readers_list[0][1][0]
        return (reader.bins, reader.hist)

In [ ]:
sequence = [(TestScribbler(), TestCollector())]

In [ ]:
Dataset = namedtuple("Dataset", "name tree files")
hzz = Dataset(name = "hzz",
             tree = "events",
             files = ["HZZ.root"])
zuu = Dataset(name = "zuu",
             tree = "events",
             files = ["Zmumu.root"])
datasets = [zuu] #,hzz]

In [ ]:
result = process.run(datasets, sequence)

In [ ]:
bins, hist = result[0][0]

In [ ]:
plt.hist(
    hist,
    bins = bins,
    log = True,
)